In [151]:
import numpy as np
import reedsolo as rs

In [152]:
messageTest = "Le KBCode est l'avenir des QRCodes!"

def KBCode(message,R="00",G="01",B="10",W="11",encoding="11",encryptionPercentage=30) :
    
    #Le KBCode à une largeur de 17 pixels pour une longeur de 2 cibles + une en-tête + le nombre de blocs divisé par 4 (4 blocs par colonnes)
    #Le nombre de blocs total est inconnu pour le moment
    #La 17 ligne sera une ligne d'alignement qui permettra en plus d'indiquer le sens du code, elle sera ajoutée à la matrice finale
    matrixCode = np.zeros((16,8+1+8))
    data1 = ["00","01","10","11",encoding,"00","00","00"]
    #print(matrixCode.shape)

    #Transforme la valeur binaire en caractère représentant la couleur
    def convertBinColor(b,R,G,B,W) :
        if b==R :
            return 'R'
        if b==G :
            return 'G'
        if b==B:
            return 'B'
        if b==W:
            return 'W'
        
    #Conversion du message en binaire
    def conversionMessage(message) :    
        def convertLetterBin(letter) :
            code=[]
            string_binary_value="{0:b}".format(letter)
            if(len(string_binary_value)%2 != 0) :
                string_binary_value = '0'+string_binary_value
            for i in range(0,len(string_binary_value),2) :
                code.append(string_binary_value[i]+string_binary_value[i+1])
            while len(code) != 4 :
                code.insert(0,"00")
            return code
        messageConverti = []
        for letter in message :
            messageConverti.append(convertLetterBin(letter))
        return messageConverti
    
    #Création des deux cibles à gauche de la matrice
    def preWriteKBCode(matrixCode) :
        target =  np.array([["11","11","11","11","11","11","11","11"],
                            ["11","00","00","00","00","00","00","11"],
                            ["11","00","11","11","11","11","00","11"],
                            ["11","00","11","00","00","11","00","11"],
                            ["11","00","11","00","00","11","00","11"],
                            ["11","00","11","11","11","11","00","11"],
                            ["11","00","00","00","00","00","00","11"],
                            ["11","11","11","11","11","11","11","11"]],dtype=str)
        target2 = target.copy()
        matrixCode=np.concatenate((target,target2),axis=0,dtype=str) 
        
        return matrixCode
    
    def addMessage(matrixCode,message,encoding) :
        #En-tête
        #Encryption taille du message
        blocNumber = len(message)*4
        binNumber = bin(blocNumber)[2:]
        if(len(binNumber)%2 != 0) :
            binNumber = '0'+binNumber
        for i in range(0,len(binNumber),2) :
            data1.append(binNumber[i]+binNumber[i+1])
        while len(data1) != 16:
            data1.insert(8,"00")
        
        #Ajout de l'en-tête gauche à la matrice
        arrayd1 = np.array(data1,dtype=str)
        matrixCode = np.insert(matrixCode,8,arrayd1,axis=1)

        #Formatting in string multiple of 16 to be a rectangle
        def addFilling(string,redundancy=False) :
            if(redundancy) : 
                fillchar= bytearray([75])
            else :
                fillchar = bytearray([0])
            (q,r) = divmod(len(string)*4,16)
            filling = int((16-r)/4)
           

            string = string + (filling*fillchar)
            return string
        

        #Création du message + redondance 
            #Reed solomon
        message = message.encode(encoding='UTF-8')
        nsym = int((len(message)*encryptionPercentage)/100)
        n = len(message)+nsym
        encodage =rs.rs_encode_msg(message,nsym)
            #Découpage en deux blocs (ajout de remplissage si non multiple de 16)
        msg = encodage[:len(message)]
        msg = addFilling(msg)
        red = encodage[-nsym:]
        red = addFilling(red, redundancy=True)
            #Conversion en pixels
        msgEnc = conversionMessage(msg)
        redEnc = conversionMessage(red)
        #Reshape en deux matrices de hauteur 16
        msgMat = np.reshape(msgEnc, (-1, 16))
        msgMat = msgMat.transpose()
        redMat = np.reshape(redEnc, (-1, 16))
        redMat = redMat.transpose()

        #Mise en place des matrices avant de les ajouter au code final
        upperHalfMsg = msgMat[0:8,:]
        lowerHalfMsg = msgMat[8:16,:]
        upperHalfRed =  redMat[0:8,:]
        lowerHalfRed =  redMat[8:16,:]
        lowerHalfData = np.concatenate((lowerHalfRed,lowerHalfMsg),axis=1)
        upperHalfData = np.concatenate((upperHalfMsg,upperHalfRed),axis=1)
        data = np.concatenate((upperHalfData,lowerHalfData),axis=0)
        #Insérer le vecteur en-tête redondant
        data=np.insert(data,8,arrayd1,axis=1)
        #Ajouter la barre d'alignement et cibles
        matrixCode = np.concatenate((matrixCode,data),axis=1)
        alignment = np.array(["11" for _ in range(len(data[0]))],dtype=str)
        data=np.insert(data,len(data[0]),alignment,axis=0)
        return matrixCode

    matrixCode = preWriteKBCode(matrixCode)
    matrixCode = addMessage(matrixCode,message,encoding)
    return matrixCode
    



In [153]:
print(KBCode(messageTest))

['11' '11' '11' '11' '11' '11' '11' '11' '11' '11' '11' '11' '11']
[['01' '01' '01' '01' '01' '01' '01' '01' '00' '01' '00' '00' '00']
 ['00' '00' '10' '11' '10' '10' '10' '01' '01' '10' '00' '00' '00']
 ['11' '00' '01' '01' '00' '10' '01' '00' '10' '01' '00' '00' '00']
 ['00' '10' '01' '00' '01' '01' '01' '10' '11' '01' '00' '00' '00']
 ['01' '01' '00' '00' '01' '01' '01' '01' '11' '01' '00' '00' '00']
 ['10' '00' '10' '10' '11' '11' '11' '00' '00' '11' '00' '00' '00']
 ['01' '00' '00' '00' '01' '00' '00' '00' '00' '00' '00' '00' '00']
 ['01' '11' '00' '00' '10' '10' '11' '11' '00' '11' '01' '01' '01']
 ['00' '00' '01' '00' '01' '01' '01' '01' '00' '00' '00' '01' '00']
 ['00' '00' '00' '10' '10' '10' '10' '10' '00' '10' '10' '10' '10']
 ['00' '00' '10' '00' '11' '01' '11' '01' '00' '00' '00' '11' '00']
 ['00' '00' '11' '00' '11' '01' '00' '01' '00' '00' '00' '11' '01']
 ['00' '00' '01' '01' '01' '01' '00' '01' '10' '01' '01' '01' '00']
 ['00' '00' '00' '00' '10' '11' '10' '10' '00' '1

array([['11', '11', '11', '11', '11', '11', '11', '11', '00', '01', '01',
        '01', '01', '01', '01', '01', '01', '00', '01', '00', '00', '00'],
       ['11', '00', '00', '00', '00', '00', '00', '11', '01', '00', '00',
        '10', '11', '10', '10', '10', '01', '01', '10', '00', '00', '00'],
       ['11', '00', '11', '11', '11', '11', '00', '11', '10', '11', '00',
        '01', '01', '00', '10', '01', '00', '10', '01', '00', '00', '00'],
       ['11', '00', '11', '00', '00', '11', '00', '11', '11', '00', '10',
        '01', '00', '01', '01', '01', '10', '11', '01', '00', '00', '00'],
       ['11', '00', '11', '00', '00', '11', '00', '11', '11', '01', '01',
        '00', '00', '01', '01', '01', '01', '11', '01', '00', '00', '00'],
       ['11', '00', '11', '11', '11', '11', '00', '11', '00', '10', '00',
        '10', '10', '11', '11', '11', '00', '00', '11', '00', '00', '00'],
       ['11', '00', '00', '00', '00', '00', '00', '11', '00', '01', '00',
        '00', '00', '01', '00', 